<a href="https://colab.research.google.com/github/antoniopenizollo/quant/blob/main/dados_cvm_dos_balan%C3%A7os.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=88ac8895c77d0071e4d2b0ee81d70f2f246a13f480fba810db857163f6475861
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [74]:
import pandas as pd
import wget
from zipfile import ZipFile
from google.colab.data_table import DataTable
import plotly.graph_objects as go

In [3]:
url_base = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'

In [4]:
arquivos_zip = []
for ano in range (2010,2021):
  arquivos_zip.append(f'dfp_cia_aberta_{ano}.zip')

arquivos_zip

['dfp_cia_aberta_2010.zip',
 'dfp_cia_aberta_2011.zip',
 'dfp_cia_aberta_2012.zip',
 'dfp_cia_aberta_2013.zip',
 'dfp_cia_aberta_2014.zip',
 'dfp_cia_aberta_2015.zip',
 'dfp_cia_aberta_2016.zip',
 'dfp_cia_aberta_2017.zip',
 'dfp_cia_aberta_2018.zip',
 'dfp_cia_aberta_2019.zip',
 'dfp_cia_aberta_2020.zip']

In [5]:
for arq in arquivos_zip:
  wget.download(url_base+arq)

In [6]:
for arq in arquivos_zip:
  ZipFile(arq,'r').extractall('CVM')

In [7]:
!mkdir DADOS

In [8]:
nomes = ['BPA_con','BPA_ind','BPP_con','BPP_ind','DFC_MD_con','DFC_MD_ind','DFC_MI_con','DFC_MI_ind','DMPL_con','DMPL_ind','DRE_con','DRE_ind','DVA_con','DVA_ind']
for nome in nomes:
  arquivo = pd.DataFrame()
  for ano in range(2010,2020):
    arquivo = pd.concat([arquivo,pd.read_csv(f'CVM/dfp_cia_aberta_{nome}_{ano}.csv',sep=';',decimal=',',encoding='ISO-8859-1')])
  arquivo.to_csv(f'DADOS/dfp_cia_aberta_{nome}_2010_2020',index=False)

In [16]:
dre = pd.read_csv('/content/DADOS/dfp_cia_aberta_DRE_ind_2010_2020')

In [ ]:
dre.head(10)

In [19]:
dre = dre[dre['ORDEM_EXERC'] == "ÚLTIMO"]

In [21]:
dre.head(3)

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
1,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.01,Receitas da Intermediação Financeira,72173904.0,S
3,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.01.01,Operações de Crédito,47391099.0,N
5,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.01.02,Operações de Arrendamento Mercantil,40372.0,N


In [29]:
empresas = dre[['DENOM_CIA','CD_CVM']].drop_duplicates().set_index('CD_CVM')
DataTable(empresas)

,DENOM_CIA
CD_CVM,
1023,BCO BRASIL S.A.
14206,BRB BCO DE BRASILIA S.A.
2437,CENTRAIS ELET BRAS S.A. - ELETROBRAS
14451,CIA ENERGETICA DE BRASILIA
14540,VBC ENERGIA SA
...,...
25054,CONSIG 6 I COMPANHIA SECURITIZADORA DE CRÉDITO...
25208,TRACK & FIELD CO S.A.
24953,"ALLPARK EMPREENDIMENTOS, PARTICIPAÇÕES E SERVI..."


In [44]:
DataTable(empresa[['CD_CONTA','DS_CONTA']].drop_duplicates().set_index('CD_CONTA'))

,DS_CONTA
CD_CONTA,
3.01,Receita de Venda de Bens e/ou Serviços
3.02,Custo dos Bens e/ou Serviços Vendidos
3.03,Resultado Bruto
3.04,Despesas/Receitas Operacionais
3.04.01,Despesas com Vendas
3.04.02,Despesas Gerais e Administrativas
3.04.03,Perdas pela Não Recuperabilidade de Ativos
3.04.04,Outras Receitas Operacionais
3.04.05,Outras Despesas Operacionais


In [45]:
conta = empresa[empresa['CD_CONTA'] == '3.99.02.02']

In [47]:
conta.index = pd.to_datetime(conta['DT_REFER'])
conta.head()

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
DT_REFER,,,,,,,,,,,,,,,
2010-12-31,61.532.644/0001-15,2010-12-31,1,ITAUSA S.A.,7617,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.99.02.02,PN,1.01,N
2011-12-31,61.532.644/0001-15,2011-12-31,1,ITAUSA S.A.,7617,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-12-31,3.99.02.02,PN,1.10,N
2012-12-31,61.532.644/0001-15,2012-12-31,1,ITAUSA S.A.,7617,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2012-01-01,2012-12-31,3.99.02.02,PN,0.94,N
2013-12-31,61.532.644/0001-15,2013-12-31,1,ITAUSA S.A.,7617,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2013-01-01,2013-12-31,3.99.02.02,PN,1.05,N
2014-12-31,61.532.644/0001-15,2014-12-31,1,ITAUSA S.A.,7617,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2014-01-01,2014-12-31,3.99.02.02,PN,1.30,N


# Calculando PL

In [51]:
!pip install yfinance

In [52]:
import yfinance as yf

In [57]:
prices = yf.download('ITSA4.SA',start='2011-01-01')[['Adj Close','Close']]

[*********************100%***********************]  1 of 1 completed


In [58]:
prices.head()

,Adj Close,Close
Date,,
2011-01-03,4.325032,7.28339
2011-01-04,4.380689,7.37712
2011-01-05,4.436355,7.47086
2011-01-06,4.341402,7.31096
2011-01-07,4.213715,7.09593


In [59]:
indicadores = prices.join(conta['VL_CONTA'],how='outer')

In [61]:
indicadores.rename({'VL_CONTA': 'LPA'},axis = 1,inplace = True)

In [62]:
indicadores

,Adj Close,Close,LPA
2010-12-31,NaN,NaN,1.01
2011-01-03,4.325032,7.28339,NaN
2011-01-04,4.380689,7.37712,NaN
2011-01-05,4.436355,7.47086,NaN
2011-01-06,4.341402,7.31096,NaN
...,...,...,...
2020-11-30,10.559999,10.58000,NaN
2020-12-01,11.040000,11.04000,NaN
2020-12-02,11.030000,11.03000,NaN
2020-12-03,11.190000,11.19000,NaN


In [63]:
 indicadores.fillna(method='ffill',inplace=True)

In [65]:
indicadores.dropna(inplace=True)

In [69]:
indicadores

,Adj Close,Close,LPA
2011-01-03,4.325032,7.28339,1.01
2011-01-04,4.380689,7.37712,1.01
2011-01-05,4.436355,7.47086,1.01
2011-01-06,4.341402,7.31096,1.01
2011-01-07,4.213715,7.09593,1.01
...,...,...,...
2020-11-30,10.559999,10.58000,1.23
2020-12-01,11.040000,11.04000,1.23
2020-12-02,11.030000,11.03000,1.23
2020-12-03,11.190000,11.19000,1.23


In [71]:
indicadores['PL_AJUSTADO'] = indicadores['Adj Close'] / indicadores['LPA']

In [72]:
indicadores['PL'] = indicadores['Close'] / indicadores['LPA']

In [73]:
indicadores

,Adj Close,Close,LPA,PL_AJUSTADO,PL
2011-01-03,4.325032,7.28339,1.01,4.282210,7.211277
2011-01-04,4.380689,7.37712,1.01,4.337316,7.304079
2011-01-05,4.436355,7.47086,1.01,4.392430,7.396891
2011-01-06,4.341402,7.31096,1.01,4.298418,7.238574
2011-01-07,4.213715,7.09593,1.01,4.171995,7.025673
...,...,...,...,...,...
2020-11-30,10.559999,10.58000,1.23,8.585365,8.601626
2020-12-01,11.040000,11.04000,1.23,8.975610,8.975610
2020-12-02,11.030000,11.03000,1.23,8.967479,8.967479
2020-12-03,11.190000,11.19000,1.23,9.097561,9.097561


In [75]:
fig = go.Figure()

In [ ]:
fig.add_trace(go.Scatter(x=indicadores.index,y=indicadores['PL'],name='PL'))

In [77]:
fig.add_trace(go.Scatter(x=indicadores.index,y=indicadores['PL'],name='PL'))
fig.add_trace(go.Scatter(x=indicadores.index,y=indicadores['PL_AJUSTADO'],name='PL_AJUSTADO'))
fig.add_trace(go.Scatter(x=indicadores.index,y=indicadores['Close'],name='Close'))
fig.add_trace(go.Scatter(x=indicadores.index,y=indicadores['Adj Close'],name='ITSA4_Ajustado'))